Reading in the data and then filtering out SNPs which do not meet p value significance before joining the data together.

In [1]:
import polars as pl

# exp: Exposure
# out: Outcome
# ea: Exposure allele
# oa: Other allele

exp_header_dict = {
  'rsID':'rsid',
  'CHROM':'chr_exp',
  'ALT':'ea_exp',
  'REF':'oa_exp',
  'POOLED_ALT_AF':'maf_exp',
  'EFFECT_SIZE':'beta_exp',
  'SE':'se_exp',
  'pvalue':'pval_exp'
}

out_header_dict = {
  'markername':'rsid',
  'chr':'chr_out',
  'bp_hg19':'pos_out',
  'effect_allele':'ea_out',
  'noneffect_allele':'oa_out',
  'effect_allele_freq':'ea_freq',
  'beta':'beta_out',
  'se_dgc':'se_out',
  'p_dgc':'pval_out'}

pthresh = 5e-8

# Renaming columns and filtering data to only include observations which fulfill significance threshold
dexp = (pl.scan_csv("dataset/ldlc_gwas.txt",separator="\t")
        .rename(exp_header_dict)
        .filter((pl.col('pval_exp') < pthresh)))
dout = (pl.scan_csv("dataset/mi_gwas.tsv",separator="\t")
        .rename(out_header_dict))

combined = (dexp.join(dout, on='rsid')
            # Convert all data to lowercase
            .with_columns(pl.col('ea_exp').str.to_lowercase())
            .with_columns(pl.col('oa_exp').str.to_lowercase())
            .with_columns(pl.col('ea_out').str.to_lowercase())
            .with_columns(pl.col('oa_out').str.to_lowercase())
            .collect())

print(combined.shape)

(75089, 26)


Harmonizing the data.

Gathering all SNPs using fowards strand with matching effect and alternate alleles between exposure and outcome.

In [2]:
forwards_same = combined.filter(((pl.col('ea_exp') == pl.col('ea_out')) & (pl.col('oa_exp') == pl.col('oa_out'))))

Gathering all SNPs using forwards strand with flipped effect and alternate alleles between exposure and outcome. The effect is then multiplied by -1.

In [3]:
forwards_flipped = (
  combined.filter(((pl.col('ea_exp') == pl.col('oa_out')) & (pl.col('oa_exp') == pl.col('ea_out'))))
  # Flip the signs of the outcome effects
  .with_columns(
    pl.col('beta_out').mul(-1),
    pl.col('ea_freq').mul(-1).add(1)
  )
)

Flipping the outcome alleles of the remaining SNPs since the remaining valid SNPs must use the reverse strand.

In [4]:
# Find cases where alleles don't match
reverse = (combined.filter(~(((pl.col('ea_exp') == pl.col('ea_out')) & (pl.col('oa_exp') == pl.col('oa_out'))) | 
                            (((pl.col('ea_exp') == pl.col('oa_out')) & (pl.col('oa_exp') == pl.col('ea_out'))))))
            # Flipping the alleles
            .with_columns(pl.col('ea_out').str.replace('a', 't'))
            .with_columns(pl.col('ea_out').str.replace('t', 'a'))
            .with_columns(pl.col('ea_out').str.replace('g', 'c'))
            .with_columns(pl.col('ea_out').str.replace('c', 'g'))
            .with_columns(pl.col('oa_out').str.replace('a', 't'))
            .with_columns(pl.col('oa_out').str.replace('t', 'a'))
            .with_columns(pl.col('oa_out').str.replace('g', 'c'))
            .with_columns(pl.col('oa_out').str.replace('c', 'g'))
)

Gathering SNPs from reverse strand which use the same alleles for exposure and outcome.

In [5]:
reverse_same = (
  reverse.filter(((pl.col('ea_exp') == pl.col('ea_out')) & (pl.col('oa_exp') == pl.col('oa_out'))))
)

Gathering SNPs from reverse strand which flipped the effect and alternate alleles. We then multiply the effect by -1.

In [6]:
reverse_flipped = (
  # Find all reversed cases
  reverse.filter(((pl.col('ea_exp') == pl.col('oa_out')) & (pl.col('oa_exp') == pl.col('ea_out'))))
  # Flip the signs of the outcome effects
  .with_columns(
    pl.col('beta_out').mul(-1),
    pl.col('ea_freq').mul(-1).add(1)
  )
)

Combining all the different cases into one dataframe.

In [7]:
print(forwards_same.shape)
print(forwards_flipped.shape)
print(reverse_same.shape)
print(reverse_flipped.shape)

# Combining all SNPs
total = pl.concat([forwards_same, forwards_flipped, reverse_same, reverse_flipped])

(8679, 26)
(66342, 26)
(1, 26)
(6, 26)


Discarding all palindromic SNPs since we are unable to determine whether the effect and outcome alleles are matching.

In [8]:
total = total.filter(
  ~(((pl.col('ea_exp') == 'a') & (pl.col('oa_exp') == 't')) |
  ((pl.col('ea_exp') == 't') & (pl.col('oa_exp') == 'a')) |
  ((pl.col('ea_exp') == 'g') & (pl.col('oa_exp') == 'c')) |
  ((pl.col('ea_exp') == 'c') & (pl.col('oa_exp') == 'g')))
)

# maf_threshold = 0.6

# palindromic = total.filter(((pl.col('ea_exp') == 'a') & (pl.col('oa_exp') == 't')) |
#   ((pl.col('ea_exp') == 't') & (pl.col('oa_exp') == 'a')) |
#   ((pl.col('ea_exp') == 'g') & (pl.col('oa_exp') == 'c')) |
#   ((pl.col('ea_exp') == 'c') & (pl.col('oa_exp') == 'g')))

# # If ea_freq is greater than threshold then we keep the SNP
# # If ea_freq is less than 1-threshold, we keep SNP after negating the effect

# print(palindromic.select(['ea_exp', 'oa_exp', 'ea_out', 'oa_out']))


Alternatively, we can use the provided harmonize function.

In [9]:
from MR.harmonize import harmonize

total = harmonize(combined)
print(total.shape)

(63712, 26)


Pruning the data based on Linkage Disequilibirum (LD).

In [10]:
from MR.ld import ld_clump

pruned_rsids = ld_clump(total['rsid'], total['pval_exp'])

processed_data = (total.join(pruned_rsids, on='rsid'))

Calculate the causal effect

In [46]:
# from MR.calculate_effect import calculate_effect, methods

# print(methods)

import polars as pl
from scipy.optimize import minimize
import numpy as np
from statistics import stdev
from scipy.stats import chi2
from sklearn.linear_model import LinearRegression


methods = [
    'inverse_variance_weighted', 
    'wald_ratio', 
    'maximum_likelihood', 
    'simple_median', 
    'weighted_median', 
    'penalised_weighted_median',
    'weighted_mode',
    'egger_regression',
    'presso'
]


def calculate_effect(data: pl.DataFrame, method: str):
    """
    Calculates causal effect using the specified method.
    List of methods can be accessed through the methods array.
    """
    if method == 'inverse_variance_weighted':
        return mr_inverse_variance_weighted(data['beta_exp'], data['beta_out'], data['se_out'])

    elif method == 'wald_ratio':
        return mr_wald_ratio(data['beta_exp'], data['beta_out'], data['se_out'])

    elif method == 'maximum_likelihood':
        return mr_maximum_likelihood(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])

    elif method == 'simple_median':
        return mr_simple_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])

    elif method == 'weighted_median':
        return mr_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])

    elif method == 'penalised_weighted_median':
        return mr_penalised_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
    
    elif method == 'egger_regression':
        return mr_egger_regression(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])


def mr_inverse_variance_weighted(beta_exp, beta_out, se_out):
    effect = (beta_exp * beta_out * se_out** -2).sum()/(beta_exp ** 2 * se_out ** -2).sum()
    se = ((beta_exp ** 2 * se_out ** -2).sum()) ** -0.5

    return {
        'effect': effect, 'se': se
    }


def mr_wald_ratio(beta_exp, beta_out, se_out):
    effect = (beta_out/beta_exp).mean()
    se = (se_out/abs(beta_exp)).mean()

    return {
        'effect': effect, 'se': se
    }


def mr_maximum_likelihood(beta_exp, beta_out, se_exp, se_out):
    n = len(beta_exp)

    def log_likelihood(param):
        return 1/2 * ((beta_exp - param[0:n])**2/se_exp**2).sum() + 1/2 * ((beta_out - param[n] * param[0:n])**2 / se_out**2).sum()

    initial = np.append((beta_exp.to_numpy()), (beta_exp*beta_out /
                                                se_out**2).sum()/(beta_exp**2/se_out**2).sum())

    res = minimize(log_likelihood, initial)
    effect = res.x[n]
    se = res.hess_inv[n, n] ** 1/2

    return {
        'effect': effect, 'se': se
    }


def weighted_median(b_iv, weights):
    beta_IV_sorted = np.sort(b_iv)
    weights_sorted = np.sort(weights)
    weights_sum = np.cumsum(weights_sorted) - 1/2 * weights_sorted
    weights_sum = weights_sum/np.sum(weights_sorted)
    below = np.max(np.where(weights_sum < 1/2))
    return beta_IV_sorted[below-1] + (beta_IV_sorted[below] - beta_IV_sorted[below-1]) * (
        1/2-weights_sum[below-1])/(weights_sum[below]-weights_sum[below-1])


def weighted_median_se(beta_exp, beta_out, se_exp, se_out, weights, nboot=1000):
    med = []
    for i in range(0, nboot):
        beta_exp_boot = np.random.normal(
            loc=beta_exp, scale=se_exp, size=len(beta_exp))
        beta_out_boot = np.random.normal(
            loc=beta_out, scale=se_out, size=len(beta_out))
        betaIV_boot = beta_out_boot/beta_exp_boot
        med.append(weighted_median(betaIV_boot, weights))

    return stdev(med)


def mr_simple_median(beta_exp, beta_out, se_exp, se_out):
    n = len(beta_exp)
    b_iv = beta_out/beta_exp
    effect = weighted_median(b_iv, np.repeat(1/n, n))
    se = weighted_median_se(beta_exp, beta_out, se_exp,
                            se_out, np.repeat(1/n, n))

    return {
        'effect': effect, 'se': se
    }


def mr_weighted_median(beta_exp, beta_out, se_exp, se_out):
    b_iv = beta_out/beta_exp
    VBj = se_out**2/beta_exp**2 + \
        beta_out**2 * se_exp**2/beta_exp**4
    effect = weighted_median(b_iv, 1/VBj)
    se = weighted_median_se(beta_exp, beta_out, se_exp, se_out, 1/VBj)

    return {
        'effect': effect, 'se': se
    }


def mr_penalised_weighted_median(beta_exp, beta_out, se_exp, se_out):
    beta_iv = beta_out/beta_exp
    beta_ivw = (beta_out*beta_exp*se_out**(-2)).sum()/(beta_exp**2*se_out**(-2)).sum()
    VBj = se_out**2/beta_exp**2 + \
        beta_out**2 * se_exp**2/beta_exp**4
    weights = 1/VBj
    bwm = mr_weighted_median(beta_exp, beta_out, se_exp, se_out)
    penalty = chi2.cdf(weights*beta_iv-bwm['effect']**2, df=1)
    penalty_weights = penalty*weights
    effect = weighted_median(beta_iv, penalty_weights)
    se = weighted_median_se(beta_exp, beta_out, se_exp, se_out, penalty_weights)

    return {
        'effect': effect, 'se': se
    }

# def mr_simple_mode(beta_exp, beta_out, se_exp, se_out):

def mr_egger_regression(beta_exp, beta_out, se_exp, se_out):
    
    def sign0(x):
        x[x==0] = -1
        return np.sign(x)
    
    # to_flip = sign0(beta_exp) == -1
    beta_out = (beta_out * sign0(beta_exp)).to_numpy().reshape((-1,1))
    beta_exp = abs(beta_exp).to_numpy().reshape((-1,1))
    model = LinearRegression().fit(beta_exp, 
                                   beta_out,
                                   sample_weight=se_out**(-2))
    
    # Code for the following is drawn from
    # https://gist.github.com/grisaitis/cf481034bb413a14d3ea851dab201d31
    def get_se():
        N = len(beta_exp)
        p = 2
        X_with_intercept = np.empty(shape=(N, p), dtype=float)
        X_with_intercept[:, 0] = 1
        X_with_intercept[:, 1:p] = beta_exp
        predictions = model.predict(beta_exp)
        residuals = beta_out - predictions
        residual_sum_of_squares = residuals.T @ residuals
        sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
        var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
        return [var_beta_hat[p_, p_] ** 0.5 for p_ in range(p)]
    
    effect = model.coef_[0][0]

    # se = get_se()[1] / min(1, model.sigma)
    se = get_se()[1]

    return {
        'effect': effect, 'se': se
    }
    

print('IVW')
result = calculate_effect(processed_data, 'inverse_variance_weighted')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

print('Wald ratio')
result = calculate_effect(processed_data, 'wald_ratio')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

print('Simple Median')
result = calculate_effect(processed_data, 'simple_median')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

print('Weighted Median')
result = calculate_effect(processed_data, 'weighted_median')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

print('Penalised Weighted Median')
result = calculate_effect(processed_data, 'penalised_weighted_median')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

print('Egger Regression')
result = calculate_effect(processed_data, 'egger_regression')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

print('Maximum Likelihood')
result = calculate_effect(processed_data, 'maximum_likelihood')
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}\n')

IVW
Effect: 0.5666555763511523
se: 0.024475735763108256

Wald ratio
Effect: 0.5137645701874517
se: 0.8410924885983558

Simple Median
Effect: 0.5909643015831363
se: 0.051917455024275916

Weighted Median
Effect: 2.3022507995814996
se: 0.13910938310717436

Penalised Weighted Median
Effect: 2.9361578831731068
se: 0.2842617327917785

Egger Regression
Effect: 0.6500241298528222
se: 0.04363211523625275

Maximum Likelihood
Effect: 0.5754979408718213
se: 0.0003119327790360706

